In [1]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Conv2D, Conv2DTranspose, Input, Activation, MaxPooling2D, Add, BatchNormalization, Reshape
from keras.models import Model
from keras.initializers import TruncatedNormal
from keras import regularizers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from glob import glob
import random
import os
import re
import cv2
from scipy.misc import imread, imresize
from sklearn.utils import shuffle

Using TensorFlow backend.


In [2]:
input_shape = (160,480,3)
vgg = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

### Build the model

In [3]:
# Reference for model architecture: https://people.eecs.berkeley.edu/~jonlong/long_shelhamer_fcn.pdf
def get_FCN8(num_classes):
    custom_init = TruncatedNormal(stddev=0.01)

    # Get the block 3 pooling layer from VGG
    pool3 = vgg.get_layer('block3_pool').output

    # Get the block 4 pooling layer from VGG
    pool4 = vgg.get_layer('block4_pool').output

    # Get the block 5 pooling layer from VGG
    pool5 = vgg.get_layer('block5_pool').output

    fc6 = Conv2D(4096, (7,7), kernel_initializer = custom_init,
               kernel_regularizer = regularizers.l2(0.01), padding='Same')(pool5)

    fc7 = Conv2D(4096, (1,1), kernel_initializer = custom_init,
               kernel_regularizer = regularizers.l2(0.01), padding='Same')(fc6)

    layer_7_1x1 = (Conv2D(num_classes, (1,1), kernel_initializer = custom_init,
                 kernel_regularizer = regularizers.l2(0.01), padding='Same'))(fc7)
    upsample1 = (Conv2DTranspose(num_classes, (4,4), strides = 2, kernel_initializer = custom_init,
               kernel_regularizer = regularizers.l2(0.01), padding='Same'))(layer_7_1x1)
    x = (BatchNormalization(axis=1))(upsample1)
    layer_4_1x1 = (Conv2D(num_classes, (1,1), kernel_initializer = custom_init,
                        kernel_regularizer = regularizers.l2(0.01), padding='Same'))(pool4)
    x = Add()([x, layer_4_1x1])
    upsample2 = (Conv2DTranspose(num_classes, (4,4), strides = 2, kernel_initializer = custom_init,
                               kernel_regularizer = regularizers.l2(0.01), padding='Same'))(x)
    x = (BatchNormalization(axis=1))(upsample2)
    layer_3_1x1 = (Conv2D(num_classes, (1,1), kernel_initializer = custom_init,
                        kernel_regularizer = regularizers.l2(0.01), padding='Same'))(pool3)
    x = Add()([x, layer_3_1x1])
    upsample3 = (Conv2DTranspose(num_classes, (16,16), strides = 8, kernel_initializer = custom_init,
                               kernel_regularizer = regularizers.l2(0.01), padding='Same'))(x)

    out_layer = (Activation('softmax'))(upsample3)

    model = Model(vgg.get_layer('input_1').output, out_layer)

    return model

In [4]:
fcn8 = get_FCN8(2)
fcn8.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 160, 480, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 160, 480, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 160, 480, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 80, 240, 64)  0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [5]:
# Compile the Model
adam = Adam(lr = 0.001)
fcn8.compile(optimizer=adam,
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [6]:
data_folder = 'data_road/training'

training_images = glob(os.path.join(data_folder, 'image_2', '*.png'))
num_samples = len(training_images)
num_training = int(num_samples*0.8)


def training_generator(data_folder, image_shape, batch_size, training = True):
    image_paths = glob(os.path.join(data_folder, 'image_2', '*.png'))[:num_training]
    label_paths = {re.sub(r'_(lane|road)_', '_', os.path.basename(path)): path
        for path in glob(os.path.join(data_folder, 'gt_image_2', '*_road_*.png'))}
    background_color = np.array([255, 0, 0])
    while 1:
        random.shuffle(image_paths)
        for batch_i in range(0, len(image_paths), batch_size):
            images = []
            gt_images = []
            for image_file in image_paths[batch_i:batch_i+batch_size]:
                gt_image_file = label_paths[os.path.basename(image_file)]

                image = imresize(imread(image_file), image_shape)
                gt_image = imresize(imread(gt_image_file), image_shape)

                gt_bg = np.all(gt_image == background_color, axis=2)
                gt_bg = np.expand_dims(gt_bg, axis=2)
                gt_image = np.concatenate((gt_bg, np.invert(gt_bg)), axis=2)

                images.append(image)
                gt_images.append(gt_image)

            yield np.array(images), np.array(gt_images)
            
def validation_generator(data_folder, image_shape, batch_size, training = True):

    image_paths = glob(os.path.join(data_folder, 'image_2', '*.png'))[num_training:]
    label_paths = {re.sub(r'_(lane|road)_', '_', os.path.basename(path)): path
        for path in glob(os.path.join(data_folder, 'gt_image_2', '*_road_*.png'))}
    background_color = np.array([255, 0, 0])
    while 1:
        random.shuffle(image_paths)
        for batch_i in range(0, len(image_paths), batch_size):
            images = []
            gt_images = []
            for image_file in image_paths[batch_i:batch_i+batch_size]:
                gt_image_file = label_paths[os.path.basename(image_file)]

                image = imresize(imread(image_file), image_shape)
                gt_image = imresize(imread(gt_image_file), image_shape)

                gt_bg = np.all(gt_image == background_color, axis=2)
                gt_bg = np.expand_dims(gt_bg, axis=2)
                gt_image = np.concatenate((gt_bg, np.invert(gt_bg)), axis=2)

                images.append(image)
                gt_images.append(gt_image)

            yield np.array(images), np.array(gt_images)

### Train the model

In [ ]:
checkpoint = ModelCheckpoint('fcn8_weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)

batch_size=8
train_generator = training_generator('data_road/training', (160,480), batch_size=batch_size)
val_generator = validation_generator('data_road/training', (160,480), batch_size=batch_size, training=False)
fcn8_history = fcn8.fit_generator(train_generator, steps_per_epoch = num_training//batch_size,
                    epochs = 25, verbose = 1, validation_data = val_generator,
                    validation_steps = (num_samples - num_training)//batch_size, callbacks=[checkpoint],
                                   initial_epoch=0)

In [ ]:
plt.plot(fcn8_history.history['val_acc'])

### Visualize some predictions

In [ ]:
test_image = imresize(imread('data_road/testing/image_2/um_000000.png'), (160,480))
plt.grid('off')
plt.imshow(test_image);

In [ ]:
pred = fcn8.predict(test_image.reshape(1,160,480,3))
mask = np.dstack((np.zeros_like(pred[0,:,:,0]),
                 np.round(pred[0,:,:,1]),
                 np.zeros_like(pred[0,:,:,0])))
plt.grid('off')
plt.imshow(mask.astype('uint8')*255);

In [ ]:
output = cv2.addWeighted(test_image, 1.0, mask.astype('uint8')*255, 0.6, 0)
plt.grid('off')
plt.imshow(output);

In [ ]:
test_image = imresize(imread('data_road/testing/image_2/umm_000002.png'), (160,480))
plt.grid('off')
plt.imshow(test_image);

In [ ]:
pred = fcn8.predict(test_image.reshape(1,160,480,3))
mask = np.dstack((np.zeros_like(pred[0,:,:,0]),
                 np.round(pred[0,:,:,1]),
                 np.zeros_like(pred[0,:,:,0])))
plt.grid('off')
plt.imshow(mask.astype('uint8')*255);

In [ ]:
output = cv2.addWeighted(test_image, 1.0, mask.astype('uint8')*255, 0.6, 0)
plt.grid('off')
plt.imshow(output);